In [ ]:
import pandas as pd

# Specify the data types for the columns
dtypes = {
    'INC NUMBER': 'str',
    'OCCURRED ON': 'str',
    'OCCURRED TO': 'str',
    'UCR CRIME CATEGORY': 'str',
    '100 BLOCK ADDR': 'str',
    'ZIP': 'float',
    'PREMISE TYPE': 'str',
    'GRID': 'str'
}

# Read the CSV file with specified data types
crime_data = pd.read_csv('./Resources/Crime-data.csv', dtype=dtypes)

# Drop rows with missing 'ZIP' values
crime_data = crime_data.dropna(subset=['ZIP'])

# Remove decimal points from the 'ZIP' column
crime_data['ZIP'] = crime_data['ZIP'].astype(int)

# Create date info base on occured to
crime_data = crime_data.dropna(subset=['OCCURRED TO'])
crime_data['OCCURRED_TO_datetime'] = pd.to_datetime(crime_data['OCCURRED TO'], errors='coerce')
crime_data['OCCURRED_TO_MMDDYY'] = crime_data['OCCURRED_TO_datetime'].dt.strftime('%m/%d/%y')

# Display the dataframe
crime_data.head(10)

,INC NUMBER,OCCURRED ON,OCCURRED TO,UCR CRIME CATEGORY,100 BLOCK ADDR,ZIP,PREMISE TYPE,GRID,OCCURRED_TO_datetime,OCCURRED_TO_MMDDYY
0,201700001603695,11/01/2015 00:00,03/31/2016 00:00,RAPE,38XX W CAMELBACK RD,85019,HOSPITAL,BI20,2016-03-31 00:00:00,03/31/16
2,201500002168686,11/01/2015 00:00,11/11/2015 09:30,LARCENY-THEFT,14XX E HIGHLAND AVE,85014,PARKING LOT,BI30,2015-11-11 09:30:00,11/11/15
3,201600000527709,11/01/2015 00:00,03/22/2016 00:36,LARCENY-THEFT,33XX W CAMELBACK RD,85017,PARKING LOT,BJ21,2016-03-22 00:36:00,03/22/16
4,201600000052855,11/01/2015 00:00,01/09/2016 00:00,MOTOR VEHICLE THEFT,N 43RD AVE & W CACTUS RD,85029,SINGLE FAMILY HOUSE,DA19,2016-01-09 00:00:00,01/09/16
5,201500002103724,11/01/2015 00:00,11/01/2015 15:21,DRUG OFFENSE,54XX W INDIAN SCHOOL RD,85031,APARTMENT,BH16,2015-11-01 15:21:00,11/01/15
6,201500002102668,11/01/2015 00:00,11/01/2015 11:50,MOTOR VEHICLE THEFT,69XX W WOOD ST,85043,SINGLE FAMILY HOUSE,AF12,2015-11-01 11:50:00,11/01/15
7,201500002101405,11/01/2015 00:00,11/01/2015 05:00,MOTOR VEHICLE THEFT,102XX W MEDLOCK AVE,85307,SINGLE FAMILY HOUSE,BJ04,2015-11-01 05:00:00,11/01/15
8,201500002102327,11/01/2015 00:00,11/01/2015 09:00,LARCENY-THEFT,51XX N 15TH ST,85014,APARTMENT,BJ30,2015-11-01 09:00:00,11/01/15
10,201500002102534,11/01/2015 00:01,11/01/2015 03:55,BURGLARY,38XX W HOLLY ST,85009,SINGLE FAMILY HOUSE,BD20,2015-11-01 03:55:00,11/01/15
11,201500002102467,11/01/2015 00:01,11/01/2015 11:15,LARCENY-THEFT,65XX N 17TH AVE,85015,VEHICLE,CB25,2015-11-01 11:15:00,11/01/15
